In [1]:
print("OK")

OK


In [2]:
%pwd

'e:\\work\\LLM project\\MedRAG\\MedRAG-Consultant\\research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'e:\\work\\LLM project\\MedRAG\\MedRAG-Consultant'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\Administrator\anaconda3\envs\MedRAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

extracted_data = load_pdf_files("data")

print(extracted_data[0])

page_content='9th Edition
Product
Guide' metadata={'producer': 'Adobe Acrobat Pro 11.0.2', 'creator': 'Adobe Acrobat Pro 11.0.2', 'creationdate': '2020-10-21T13:41:02+06:00', 'moddate': '2020-10-21T16:43:52+06:00', 'source': 'data\\data.pdf', 'total_pages': 353, 'page': 0, 'page_label': '1'}


In [7]:
extracted_data = load_pdf_files("data")

In [8]:
extracted_data

[Document(metadata={'producer': 'Adobe Acrobat Pro 11.0.2', 'creator': 'Adobe Acrobat Pro 11.0.2', 'creationdate': '2020-10-21T13:41:02+06:00', 'moddate': '2020-10-21T16:43:52+06:00', 'source': 'data\\data.pdf', 'total_pages': 353, 'page': 0, 'page_label': '1'}, page_content='9th Edition\nProduct\nGuide'),
 Document(metadata={'producer': 'Adobe Acrobat Pro 11.0.2', 'creator': 'Adobe Acrobat Pro 11.0.2', 'creationdate': '2020-10-21T13:41:02+06:00', 'moddate': '2020-10-21T16:43:52+06:00', 'source': 'data\\data.pdf', 'total_pages': 353, 'page': 1, 'page_label': '2'}, page_content='or  gene\n \nPreface\nIt is our immense pleasure to present the 9th edition of ‘’Product \nGuide’’. The focus of this book is to provide information on SQUARE’s \nproducts. This compilation is enriched with abridged prescribing \ninformation of the products of SQUARE Pharmaceuticals Ltd. \nwhich includes active ingredient, indication, dosage & administration,\npreparation.\n \n‘product name wise index’ ric name 

In [9]:
len(extracted_data)

353

In [10]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [11]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [12]:
minimal_docs

[Document(metadata={'source': 'data\\data.pdf'}, page_content='9th Edition\nProduct\nGuide'),
 Document(metadata={'source': 'data\\data.pdf'}, page_content='or  gene\n \nPreface\nIt is our immense pleasure to present the 9th edition of ‘’Product \nGuide’’. The focus of this book is to provide information on SQUARE’s \nproducts. This compilation is enriched with abridged prescribing \ninformation of the products of SQUARE Pharmaceuticals Ltd. \nwhich includes active ingredient, indication, dosage & administration,\npreparation.\n \n‘product name wise index’ ric name wise index’.\n \nbook useful in their daily practice.\n \nWe believe that we would be able to keep on our endeavor to bring \ncontinual improvement with the support of our valued doctors.\n \n \nSincerely\nProduct Management Department\nSQUARE Pharmaceuticals Ltd.\ncontraindication & precaution,'),
 Document(metadata={'source': 'data\\data.pdf'}, page_content=''),
 Document(metadata={'source': 'data\\data.pdf'}, page_content

In [13]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [14]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 1873


In [15]:
texts_chunk

[Document(metadata={'source': 'data\\data.pdf'}, page_content='9th Edition\nProduct\nGuide'),
 Document(metadata={'source': 'data\\data.pdf'}, page_content='or  gene\n \nPreface\nIt is our immense pleasure to present the 9th edition of ‘’Product \nGuide’’. The focus of this book is to provide information on SQUARE’s \nproducts. This compilation is enriched with abridged prescribing \ninformation of the products of SQUARE Pharmaceuticals Ltd. \nwhich includes active ingredient, indication, dosage & administration,\npreparation.\n \n‘product name wise index’ ric name wise index’.\n \nbook useful in their daily practice.'),
 Document(metadata={'source': 'data\\data.pdf'}, page_content='We believe that we would be able to keep on our endeavor to bring \ncontinual improvement with the support of our valued doctors.\n \n \nSincerely\nProduct Management Department\nSQUARE Pharmaceuticals Ltd.\ncontraindication & precaution,'),
 Document(metadata={'source': 'data\\data.pdf'}, page_content='Pro

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9492\30168171.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [17]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
vector = embedding.embed_query("Hello world")
vector

[-0.03447731211781502,
 0.03102317824959755,
 0.006734973285347223,
 0.026108987629413605,
 -0.03936202451586723,
 -0.16030248999595642,
 0.06692396104335785,
 -0.006441520527005196,
 -0.047450557351112366,
 0.014758833684027195,
 0.0708753690123558,
 0.05552764981985092,
 0.019193340092897415,
 -0.026251304894685745,
 -0.010109521448612213,
 -0.026940487325191498,
 0.022307438775897026,
 -0.022226588800549507,
 -0.1496925950050354,
 -0.017493003979325294,
 0.007676261477172375,
 0.054352227598428726,
 0.003254449460655451,
 0.031725913286209106,
 -0.0846213772892952,
 -0.029406029731035233,
 0.05159557983279228,
 0.0481240414083004,
 -0.0033148375805467367,
 -0.05827924236655235,
 0.041969284415245056,
 0.02221062034368515,
 0.1281888782978058,
 -0.0223389845341444,
 -0.01165628433227539,
 0.06292835623025894,
 -0.03287626802921295,
 -0.09122604876756668,
 -0.031175415962934494,
 0.052699532359838486,
 0.047034814953804016,
 -0.08420310914516449,
 -0.030056163668632507,
 -0.0207448359

In [19]:
print( "Vector length:", len(vector))

Vector length: 384


In [20]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [21]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [22]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [23]:
pc

In [24]:
from pinecone import ServerlessSpec 

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [25]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [26]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [27]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [28]:
retrieved_docs = retriever.invoke("what is the dosage of paracetamol")
retrieved_docs

[Document(id='2633c222-f4b3-470b-b841-54d972c00b10', metadata={'source': 'data\\data.pdf'}, page_content='Dosage & Administration\nAdults & children above 6 years of age:\n1 tablet daily (Preferably in the evening). The \nduration of treatment should be 5-10 days in \nacute treatment.\nIn Paracetamol overdose:\nInitially 140 mg/kg, followed by 70 mg/kg every \n4 hours for an additional 17 doses.\nDissolve 1 tablet in a glass of water (200 ml) to \ndrink\nContraindication & Precaution\nKnown hypersensitivity to active ingredient. \nAlso contraindicated in patients suffering from'),
 Document(id='e21b94d3-5c3e-4ef2-b247-baa34f8f3758', metadata={'source': 'data\\data.pdf'}, page_content='months: 1.0 ml (80 mg). 1 to 2 years: 1.5 ml (120 \nmg) Dose can be repeated, every 4 hours. \nSuppository: Suppository should be \nadministered rectaly. Children 3 months - 1 \nyear : 60-120 mg 4 times daily. Children below \n5 years : 125-250 mg, 4 times daily. Children \n6-12 years  : 250-500 mg, 4 tim

In [29]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

# Defining the LLM model
chatModel = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.4
)

c:\Users\Administrator\anaconda3\envs\MedRAG\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [30]:
# Defining Prompt
prompt_template = """
Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer:
"""

In [31]:
# Creating the prompt template object
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [32]:
# Creating the RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=chatModel,
    chain_type="stuff",
    retriever=retriever,
    input_key="input",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [37]:
# Testing the RAG chain
response = rag_chain.invoke({"input": "what is generic name of ace"})
print(response["result"])

The generic name of Ace is Paracetamol.


In [42]:
# Testing the RAG chain
response = rag_chain.invoke({"input": "I am suffering from fever. What medication should I take?"})
print(response["result"])

Based on the context, if you are suffering from fever, you could consider taking **Ace Power**, which contains **Paracetamol BP 1000 mg**.


In [44]:
# Testing the RAG chain
response = rag_chain.invoke({"input": "Cough syrup for children"})
print(response["result"])

I don't know. The context describes Brofex, which is for cough and has dosages for children, but its preparation is a "solution for infusion," not a syrup. Another medicine is mentioned with a "Syrup" preparation and dosage for "Children 6 years & older," but its indication is "allergen induced asthma," not cough.


In [45]:
# Testing the RAG chain
response = rag_chain.invoke({"input": "What is the use of Rosuva?"})
print(response["result"])

The use of Rosuva is for Hypercholesterolemia.
